In [6]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.Collecting langchain==0.0.284 (from -r requirements.txt (line 1))
  Using cached langchain-0.0.284-py3-none-any.whl.metadata (14 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl.metadata (21 kB)
  Using cached streamlit-1.22.0-py2.py3-none-any.whl.metadata (7.3 kB)
  Using cached tiktoken-0.4.0.tar.gz (25 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'



ERROR: Ignored the following versions that require a different python version: 0.55.2 Requires-Python <3.5
ERROR: Could not find a version that satisfies the requirement faiss-cpu==1.7.4 (from versions: 1.8.0, 1.8.0.post1, 1.9.0, 1.9.0.post1, 1.10.0, 1.11.0, 1.11.0.post1, 1.12.0, 1.13.0, 1.13.1)

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for faiss-cpu==1.7.4


In [1]:
%pip install langchain-huggingface huggingface_hub


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:

from dotenv import load_dotenv
import os
load_dotenv("E:\\AI-Buildathon\\.env")
api_key = os.getenv("GOOGLE_GENAI_API_KEY")




In [17]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7, api_key=api_key)





In [6]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def load_pdf_documents(file_path):
    loader= DirectoryLoader(
        file_path,
        glob="**/*.pdf",
        loader_cls=PyPDFLoader,
    )
    documents=loader.load()
    return documents

document=load_pdf_documents("E:\\AI-Buildathon\\data")


document


[Document(metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Windows)', 'creationdate': '2019-09-30T15:52:18+06:00', 'moddate': '2019-10-28T11:22:49+06:00', 'trapped': '/False', 'source': 'E:\\AI-Buildathon\\data\\2025-04-10-10-53-bad53858a0ac0faccce06c86d7a65f77.pdf', 'total_pages': 90, 'page': 0, 'page_label': 'a'}, page_content='a\nBangladesh National Strategy for Maternal Health 2019-2030\n\x03\n%DQJODGHVK\x031DWLRQDO\n6WUDWHJ\\\x03IRU\n0DWHUQDO\x03+HDOWK\nGovernment of the People’s Republic of Bangladesh\nMinistry of Health and Family Welfare\n\x15\x13\x14\x1c\x10\x15\x13\x16\x13'),
 Document(metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Windows)', 'creationdate': '2019-09-30T15:52:18+06:00', 'moddate': '2019-10-28T11:22:49+06:00', 'trapped': '/False', 'source': 'E:\\AI-Buildathon\\data\\2025-04-10-10-53-bad53858a0ac0faccce06c86d7a65f77.pdf', 'total_pages': 90, 'page': 1, 'page_label': 'b'}, page_content='b\nBangla

In [5]:
for doc in document:
    doc.metadata.update({
        "source": ["WHO","DGHS","The pregnancy encyclopedia","The Gale encyclopedia of medicine"],
        "category": ["disease","symptom","treatment","prevention","diagnosis","Health Tips","Nutrition","Exercise","Mental Health","Medications","Vaccinations"],
        "region": "Bangladesh",
        "language": "en",
        "confidence_level": "primary"
    })


In [7]:
def text_splitter(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=500,
        length_function=len,
    )
    text_chunk = text_splitter.split_documents(document)
    return text_chunk

text_chunk= text_splitter(document)
print(f"Number of Chunks: {len(text_chunk)}")

Number of Chunks: 8164


In [8]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")




In [9]:
load_dotenv("E:\\AI-Buildathon\\.env")
pinecone_api_key = os.getenv("PINECONE_API_KEY")

In [10]:
from pinecone import Pinecone

pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index("test")

In [11]:
from langchain_pinecone import PineconeVectorStore

docstore = PineconeVectorStore.from_documents(
    documents=text_chunk,
    embedding=embedding_model,
    index_name="test",
)


In [12]:
docsearch= PineconeVectorStore.from_existing_index(
    embedding=embedding_model,
    index_name="test"
)

In [13]:
retriever=docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})


In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import (create_stuff_documents_chain)

system_prompt=(
    "You are MaBondhu AI, a clinical assistant for maternal health. "
        "Provide safe, evidence-based, detailed guidance. Use the following pieces of retrieved context to answer the question at the end. "
        "Language: {language}.\n\n"
        "Rules:\n"
        "1) If this is potentially life-threatening, say 'EMERGENCY' and recommend immediate referral.\n"
        "2) If unsure, recommend human-in-the-loop evaluation and list needed vitals/tests.\n"
        "3) If the question is about nutrition, lifestyle, or common symptoms,provide a clear, helpful answer.\n"
        "4) If the question is about giving advices or tips during pregnancy, postpartum, or newborn care, provide a clear, helpful answer.\n"
        "5) ONLY recommend consulting a doctor if the context mentions a danger sign or emergency.\n"
        "6) Include 1–2 specific, actionable next steps (e.g., 'go to nearest clinic', 'take iron supplement 60mg').\n"
        "7) Provide detailed Bangla/English text depending on language.\n\n"
        "8) If you don't know the answer, say 'Sorry, I don't know about it'.\n\n"
        "Be practical, culturally appropriate, and reassuring.\n\n"
        "Answer now in clear {language} with citations (e.g., [WHO Section 3])."
        "\n\n"
        "Context:\n{context}\n\n"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

In [19]:
question_answering_chain = create_stuff_documents_chain(llm,prompt=prompt)
rag_chain = create_retrieval_chain(retriever, question_answering_chain)



In [33]:

inp=input("Enter your question: ")
response = rag_chain.invoke({
    "input": inp,
    "language": {"English","Bengali" }
})

print(response['answer'])  

While some traditional remedies can be helpful, it's crucial to be cautious during pregnancy as not all are safe.

**English:**
**Herbal Teas:** Many herbal teas have properties that are not always safe for pregnancy. It is best to choose a specialist “pregnancy herbal tea” which comes pre-blended. Individual herbs like mint and lemon balm are considered safe for drinking during pregnancy and can help with nausea and anxiety [HERBAL TEAS]. Therefore, avoid other general herbal teas unless specifically confirmed safe by a healthcare professional or if they are specialist pregnancy teas.

**Alcohol:** Alcohol crosses the placenta, meaning any alcohol you drink can affect your baby. Therefore, alcohol should be completely avoided during pregnancy [ALCOHOL].

**Safe Options for Nausea:** For relief from nausea in early pregnancy, options like ginger and chamomile are recommended and are unlikely to have harmful effects on the mother and baby [WHO recommendations].

**Next Steps:**
1.  Alwa